## Câu 1: viết câu lệnh sql

In [58]:
import sqlite3
# Câu SQL tính hệ số tương quan Pearson
query = """
SELECT 
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) AS numerator,
    SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) AS denominator_a,
    SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B)) AS denominator_b
FROM data
"""

## Câu 2: thử nghiệm và chuyển đổi dữ liệu

In [68]:
from scipy.stats import f_oneway
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Tạo bảng dữ liệu đầy đủ
data = {
    'Day': ['D1']*3 + ['D2']*3 + ['D3']*3 + ['D4']*3,
    'Model': ['A', 'B', 'C']*4,
    'Score': [8, 9, 7, 7.5, 8.5, 7, 6, 7, 8, 7, 6, 5]
}
df = pd.DataFrame(data)
print(df)
# Dữ liệu điểm của từng mẫu
A1 = [8, 7.5, 6, 7]
A2= [9, 8.5, 7, 6]
A3= [7, 7, 8, 5]

# Kiểm định ANOVA một chiều
f_stat, p_value = f_oneway(A1, A2, A3)

print(f"\nF-statistic = {f_stat:.4f}")
print(f"P-value = {p_value:.4f}")


   Day Model  Score
0   D1     A    8.0
1   D1     B    9.0
2   D1     C    7.0
3   D2     A    7.5
4   D2     B    8.5
5   D2     C    7.0
6   D3     A    6.0
7   D3     B    7.0
8   D3     C    8.0
9   D4     A    7.0
10  D4     B    6.0
11  D4     C    5.0

F-statistic = 0.5495
P-value = 0.5954


Kết luận: 
- P-value = 0.5954 > 0.05 nên không đủ bằng chứng để kết luận có sự khác biệt đáng kể giữa các mẫu ở mức ý nghĩa 5%.
- Chưa đủ mạnh để khẳng định các mẫu khác nhau đáng kể, nhưng mẫu B nhìn chung có điểm cao hơn, mẫu C có vẻ yếu hơn.

In [69]:
# Chạy ANOVA hai chiều
model = ols('Score ~ Day + Model', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)


            sum_sq   df         F    PR(>F)
Day       7.000000  3.0  2.488889  0.157555
Model     1.541667  2.0  0.822222  0.483522
Residual  5.625000  6.0       NaN       NaN


- P-value của Day = 0.157 > 0.05, không có đủ bằng chứng để kết luận điểm số khác biệt theo từng ngày.
- P-value của Model = 0.4835 > 0.05, không có đủ bằng chứng để kết luận điểm số khác biệt theo từng mẫu xe.

Kết luận: không có yếu tố nào (cả ngày lẫn mẫu xe) ảnh hưởng đáng kể đến kết quả điểm số.

In [72]:
# chuyển đổi dữ liệu sang dạng quan hệ
from scipy.stats import chi2_contingency
# Phân loại điểm số
def score_level(score):
    if score >= 8:
        return 'Cao'
    elif score >= 6.5:
        return 'Trung bình'
    else:
        return 'Thấp'

df['Level'] = df['Score'].apply(score_level)

# Tạo bảng chéo (contingency table)
table = pd.crosstab(df['Model'], df['Level'])
print("Bảng chéo:\n", table)

# Kiểm định Chi-squared
chi2, p, dof, expected = chi2_contingency(table)

print(f"\nChi-squared = {chi2:.4f}")
print(f"P-value = {p:.4f}")
print(f"Mức độ tự do = {dof}")
print("\nTần suất dự kiến:\n", pd.DataFrame(expected, index=table.index, columns=table.columns))
# Đánh giá kết quả kiểm định
if p < 0.05:
    print(" => Có sự khác biệt đáng kể giữa các mẫu xe (bác bỏ H0).")
else:
    print(" => Không có sự khác biệt đáng kể giữa các mẫu xe (chấp nhận H0).")


Bảng chéo:
 Level  Cao  Thấp  Trung bình
Model                       
A        1     1           2
B        2     1           1
C        1     1           2

Chi-squared = 0.9000
P-value = 0.9246
Mức độ tự do = 4

Tần suất dự kiến:
 Level       Cao  Thấp  Trung bình
Model                            
A      1.333333   1.0    1.666667
B      1.333333   1.0    1.666667
C      1.333333   1.0    1.666667
 => Không có sự khác biệt đáng kể giữa các mẫu xe (chấp nhận H0).


## Câu 3
### Bảng flights(departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30 AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian

In [ ]:
import sqlite3
import pandas as pd

# Hàm nhập tay dữ liệu
def nhap_du_lieu():
    data = []
    while True:
        user_input = input("Nhập thời gian dạng HHMM (gõ 'done' để kết thúc): ")
        if user_input.lower() == 'done':
            break
        try:
            t = int(user_input)
            if t < 1 or t > 2359:
                continue
            hour = t // 100
            minute = t % 100
            if hour >= 24 or minute >= 60:
                print("Giờ phải < 24 và phút < 60.")
                continue
            data.append((t,))
        except ValueError:
            print("Nhập số nguyên hợp lệ (từ 1 đến 2359) hoặc 'done'.")
    return data

# Hàm tạo DB và bảng flights
def tao_bang(conn, data):
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE flights (departure_time INTEGER)')
    cursor.executemany('INSERT INTO flights (departure_time) VALUES (?)', data)
    conn.commit()

# Hàm chạy SQL để chuyển định dạng
def chuyen_dinh_dang(conn):
    query = '''
    SELECT
      departure_time,
      printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
    FROM flights
    '''
    return pd.read_sql_query(query, conn)

# === Chạy chương trình ===
du_lieu = nhap_du_lieu()

if du_lieu:
    conn = sqlite3.connect(':memory:')
    tao_bang(conn, du_lieu)
    ket_qua = chuyen_dinh_dang(conn)

    print("\nKết quả chuyển đổi thời gian")
    print(ket_qua.to_string(index=False))
else:
    print("Không có dữ liệu nào được nhập.")



Kết quả chuyển đổi thời gian
 departure_time formatted_time
            453          04:53
            123          01:23


## Câu 4: 
### Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.


In [ ]:
import sqlite3
import pandas as pd

# Bước 1: Nhập tay dữ liệu
data = []
while True:
    user_input = input("Nhập giá trị (gõ 'done' để kết thúc): ")
    if user_input.lower() == 'done':
        break
    try:
        data.append((float(user_input),))
    except ValueError:
        print("Giá trị không hợp lệ!")

# Bước 2: Tạo SQLite in-memory DB và bảng
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.execute('CREATE TABLE flights (data REAL)')
cursor.executemany('INSERT INTO flights (data) VALUES (?)', data)
conn.commit()

# Bước 3: Truy vấn MAD & phát hiện ngoại lệ
stats_query = '''
WITH median_val AS (
  SELECT
    (SELECT data FROM flights ORDER BY data LIMIT 1 OFFSET (SELECT COUNT(*) FROM flights) / 2) AS median
),
abs_dev AS (
  SELECT ABS(f.data - m.median) AS abs_deviation
  FROM flights f, median_val m
),
mad_val AS (
  SELECT
    (SELECT abs_deviation FROM abs_dev ORDER BY abs_deviation LIMIT 1 OFFSET (SELECT COUNT(*) FROM abs_dev) / 2) AS mad
)
SELECT median_val.median, mad_val.mad FROM median_val, mad_val;
'''
stats = pd.read_sql_query(stats_query, conn)
median = stats['median'][0]
mad = stats['mad'][0]

print(f"\nMedian: {median}")
print(f"MAD: {mad}")

# Truy vấn dữ liệu với đánh dấu outlier
outlier_query = f'''
WITH median_val AS (
  SELECT {median} AS median
),
mad_val AS (
  SELECT {mad} AS mad
)
SELECT 
  f.data,
  CASE 
    WHEN ABS(f.data - median_val.median) > 1.5 * mad_val.mad THEN 'True'
    ELSE 'False'
  END AS is_outlier
FROM flights f, median_val, mad_val;
'''
# Bước 4: Chạy truy vấn và in kết quả
result_df = pd.read_sql_query(outlier_query, conn)
print("\nKết quả phát hiện ngoại lệ")
print(result_df)


Median: 3.0
MAD: 1.0

Kết quả phát hiện ngoại lệ
   data is_outlier
0   1.0       True
1   2.0      False
2   3.0      False
3  56.0       True
4  45.0       True
5   2.0      False
6   3.0      False


## Câu 5: 
### Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”.

In [ ]:
import sqlite3
import pandas as pd

# Bước 1: Nhập dữ liệu
patients = []
while True:
    last_name = input("Họ (last_name): ")
    if last_name.lower() == 'done':
        break
    try:
        weight = float(input("Cân nặng (kg): "))
        height = float(input("Chiều cao (cm): "))
        patients.append((last_name, weight, height))
    except ValueError:
        print("Dữ liệu không hợp lệ! Nhập lại.")

# Bước 2: Tạo SQLite DB và bảng
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE Patient (
    last_name TEXT,
    weight REAL,
    height REAL
)
''')
cursor.executemany('INSERT INTO Patient VALUES (?, ?, ?)', patients)
conn.commit()

# Bước 3: Truy vấn so sánh từng cặp
query = '''
SELECT
  p1.last_name AS name1,
  p2.last_name AS name2,
  p1.weight AS weight1,
  p2.weight AS weight2,
  p1.height AS height1,
  p2.height AS height2,
  CASE
    WHEN p1.last_name = p2.last_name
         AND ABS(p1.weight - p2.weight) <= 2
         AND p1.rowid < p2.rowid
    THEN 'Same person'
    ELSE 'Different'
  END AS match_result
FROM Patient p1
JOIN Patient p2
  ON p1.rowid < p2.rowid
'''

# Bước 4: In kết quả
df = pd.read_sql_query(query, conn)
print("\nKết quả so sánh các cặp")
print(df if not df.empty else "Không có đủ dữ liệu để so sánh.")



Kết quả so sánh các cặp
  name1 name2  weight1  weight2  height1  height2 match_result
0   Đậu   Đậu     34.0     34.0    134.0    134.0  Same person
